## Setup

### 🛠️ Imports

In [1]:
from __future__ import annotations
from aiworkshop_utils.standardlib_imports import os, json, base64, logging, Optional, List, Literal, pprint, glob, asyncio, datetime, date, time, timezone, ZoneInfo, uuid, dataclass
from aiworkshop_utils.thirdparty_imports import AutoTokenizer, load_dotenv, requests, BaseModel, Field, pd, cosine_similarity, plt, np, DataType, MilvusClient, DDGS, rprint
from aiworkshop_utils.custom_utils import show_pretty_json, encode_image
from aiworkshop_utils.jupyter_imports import Markdown, HTML, JSON, display, widgets
from aiworkshop_utils.openai_imports import OpenAI, Agent, Runner, InputGuardrail, GuardrailFunctionOutput, InputGuardrailTripwireTriggered, OpenAIChatCompletionsModel, AsyncOpenAI, set_tracing_disabled, ModelSettings, function_tool, trace, ResponseContentPartDoneEvent, ResponseTextDeltaEvent, RawResponsesStreamEvent, TResponseInputItem, ItemHelpers, MessageOutputItem, RunContextWrapper, input_guardrail, output_guardrail
from aiworkshop_utils import config

In [2]:
# Standard library imports
import os
import json
import base64
import logging
from typing import Optional, List, Dict, Any, Union, Literal
import asyncio
from datetime import datetime, date, time, timezone
from zoneinfo import ZoneInfo
import uuid
from dataclasses import dataclass
from pprint import pprint
from glob import glob

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import requests
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from rich import print as rprint
from tqdm import tqdm
from pymilvus import DataType, MilvusClient, Collection
from duckduckgo_search import DDGS

# Document processing imports
from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend
from docling.chunking import HybridChunker
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import (
    AcceleratorDevice,
    AcceleratorOptions,
    PdfPipelineOptions,
)
from docling.document_converter import (
    DocumentConverter,
    PdfFormatOption,
    WordFormatOption,
)
from docling.pipeline.simple_pipeline import SimplePipeline

# OpenAI and Agent imports
from openai import OpenAI, AsyncOpenAI
from agents import (
    Agent,
    GuardrailFunctionOutput,
    InputGuardrail,
    InputGuardrailTripwireTriggered,
    ModelSettings,
    OpenAIChatCompletionsModel,
    Runner,
    function_tool,
    set_tracing_disabled,
    trace,
    RawResponsesStreamEvent,
    ItemHelpers, 
    MessageOutputItem,
    RunContextWrapper,
    input_guardrail,
    output_guardrail
)

### Pydantic Model

In [3]:
class HealthData(BaseModel):
    age: int = Field(description="Age in years")
    gender: str = Field(description="e.g. male, female, diverse")
    weight: float = Field(description="Weight in kg")   
    height: float = Field(description="Height in cm")
    allergies: Optional[str] = Field(description="e.g. nuts, gluten")
    eating_habits: str = Field(description="e.g. vegetarian, vegan, omnivore, paleo")
    goal: str = Field(description="e.g. weight loss, muscle gain, maintenance")
    activity_level: str = Field(description="e.g. sedentary, lightly active, moderately active, very active")
    timeCooking: int = Field(description="Time spent cooking per day in minutes")
    healthCondition: Optional[str] = Field(description="e.g. diabetes, hypertension")

In [4]:
# Beispiel: Formularinstanz erzeugen und ausgeben
sample_form = HealthData(
    age=30,
    gender="männlich",
    weight=85.0,
    height=180.0,
    allergies="keine",
    eating_habits="omnivor",
    goal="Muskelaufbau",
    activity_level="sportlich",
    timeCooking=45,
    healthCondition="leicht erhöhter Blutdruck"
)

prompt1 = """
Ich bin 20 Jahre alt, männlich, wiege 85kg und bin 1,80m groß. 
Ich bin laktoseintolerant und esse gerne vegan. 
Mein Ziel ist es, mehr Muskeln aufzubauen, weshalb ich moderat Sport betreibe. 
Für das tägliche Kochen plane ich maximal 2 Stunden (120 min) ein. 
Leider bin ich auch Diabetiker, weshalb ich auch auf meinen Zuckerhaushalt achten muss. 
Welche Diätform kannst du mir empfehlen?"""

## Document

### Parsing

In [5]:
class DocumentParser:
    def __init__(self, converter):
        self.converter = converter

    def parse(self, file_path: str, options: dict = None):
        print(f"Converting document: {file_path}")
        result = self.converter.convert(file_path)
        return result.document

In [6]:
source = "assets/health_data.pdf"

my_processor = DocumentParser(DocumentConverter())
processor_result = my_processor.parse(source)

Converting document: assets/health_data.pdf


### Chunking

In [7]:
class DocumentChunker:
    def __init__(self, tokenizer: str = "sentence-transformers/all-MiniLM-L6-v2"):
        self.tokenizer = tokenizer
        self.chunker = HybridChunker(tokenizer=tokenizer)

    def chunk(self, document, options: dict = None):
        print("Chunking document...")
        chunks = list(self.chunker.chunk(document))
        print(f"Created {len(chunks)} chunks")
        return chunks

In [8]:
my_chunker = DocumentChunker()
chunker_result = my_chunker.chunk(processor_result)

Chunking document...
Created 98 chunks


### Embedding

In [9]:
class DocumentEmbedder:
    def __init__(self, url, model_name):
        self.url = url
        self.model_name = model_name

    def _post_request(self, texts):
        if isinstance(texts, str):
            texts = [texts]
        response = requests.post(
            self.url,
            json={
                "model": self.model_name,
                "input": texts
            }
        )
        response.raise_for_status()
        return response.json()

    def get_embeddings(self, texts):
        response = self._post_request(texts)
        embeddings = response["embeddings"]
        if isinstance(texts, str) or len(embeddings) == 1:
            return embeddings[0]
        return embeddings

    def get_full_response(self, texts):
        return self._post_request(texts)

    def embed(self, chunks):
        texts = [chunk.text for chunk in chunks]
        return self.get_embeddings(texts)
    
    def get_prepared_data_for_indexing(self, chunks):
        embedding_result = self.embed(chunks)
        data = []
        for chunk, vector in zip(chunks, embedding_result):
            headings = ""
            page_info = ""
            if hasattr(chunk, "meta") and chunk.meta:
                headings_list = getattr(chunk.meta, "headings", [])
                if headings_list:
                    headings = " > ".join(headings_list)
                page_info = getattr(chunk.meta, "page_info", "")
            data.append({
                "vector": vector,
                "text": chunk.text,
                "headings": headings,
                "page_info": page_info
            })
        return data

In [10]:
my_embedder = DocumentEmbedder(url=config.OAPI_EMBED_URL, model_name=config.OMODEL_NOMIC)
embedding_result = my_embedder.embed(chunker_result)

In [11]:
print(len(embedding_result))
print(embedding_result[0])

98
[0.029407937, -0.007725332, -0.16320756, -0.026395265, 0.03437459, -0.03793293, -0.049199972, -0.0077174124, -0.021334266, -0.021033404, -0.03016901, -8.664924e-05, 0.07509571, 0.085213125, 0.008133813, -0.010943744, -0.016439993, -0.043249737, -0.024111161, 0.010377481, 0.05121823, 0.03195553, -0.009324543, -0.0596079, 0.025305636, 0.0256475, -0.0048430045, 0.002121591, -0.04049873, -0.0050582634, 0.0074854787, 0.00644616, 0.0057679694, -0.00082253775, 7.414384e-05, -0.075304456, 0.016309405, 0.005631348, -0.024207316, 0.015241426, 0.02103698, -0.028154803, 0.001779003, -0.060931705, -0.008009651, 0.0022962943, -0.04469948, -0.0023547928, 0.07105846, -0.06433149, 0.0021380114, 0.043761663, -0.015382296, 0.020869996, 0.078693844, -0.036903802, -0.07706409, -0.05066658, -0.013687871, -0.053277984, 0.021080783, 0.078174956, -0.065949656, 0.014262262, 0.034166154, -0.026743347, -0.06996673, 0.056965724, 0.008793368, 0.0023380641, 0.0127667915, -0.009422646, 0.035293605, 0.0011332023, 0

In [12]:
prepared_embeddings = my_embedder.get_prepared_data_for_indexing(chunker_result)
print(prepared_embeddings[0])

{'vector': [0.029407937, -0.007725332, -0.16320756, -0.026395265, 0.03437459, -0.03793293, -0.049199972, -0.0077174124, -0.021334266, -0.021033404, -0.03016901, -8.664924e-05, 0.07509571, 0.085213125, 0.008133813, -0.010943744, -0.016439993, -0.043249737, -0.024111161, 0.010377481, 0.05121823, 0.03195553, -0.009324543, -0.0596079, 0.025305636, 0.0256475, -0.0048430045, 0.002121591, -0.04049873, -0.0050582634, 0.0074854787, 0.00644616, 0.0057679694, -0.00082253775, 7.414384e-05, -0.075304456, 0.016309405, 0.005631348, -0.024207316, 0.015241426, 0.02103698, -0.028154803, 0.001779003, -0.060931705, -0.008009651, 0.0022962943, -0.04469948, -0.0023547928, 0.07105846, -0.06433149, 0.0021380114, 0.043761663, -0.015382296, 0.020869996, 0.078693844, -0.036903802, -0.07706409, -0.05066658, -0.013687871, -0.053277984, 0.021080783, 0.078174956, -0.065949656, 0.014262262, 0.034166154, -0.026743347, -0.06996673, 0.056965724, 0.008793368, 0.0023380641, 0.0127667915, -0.009422646, 0.035293605, 0.00113

### Vektor-DB

In [13]:
class VectorDBCreator:
    def __init__(self, milvus_client_name):
        self.milvus_client = MilvusClient(f"{milvus_client_name}.db")

    def create_collection(self, collection_name: str, dimension: int, **kwargs):
        if self.milvus_client.has_collection(collection_name=collection_name):
            self.milvus_client.drop_collection(collection_name=collection_name)
        self.milvus_client.create_collection(
            collection_name=collection_name,
            dimension=dimension,
            primary_field_name='id',
            id_type=DataType.INT64,
            vector_field_name='vector',
            extra_fields=[
                {"name": "text", "type": DataType.VARCHAR, "max_length": 1024},
                {"name": "headings", "type": DataType.VARCHAR, "max_length": 512},
                {"name": "page_info", "type": DataType.VARCHAR, "max_length": 128}
            ],
            metric_type='IP',
            auto_id=True,
            consistency_level='Strong',
            **kwargs
        )
        print(f"Collection '{collection_name}' with dimension {dimension} created.")
    
    def get_milvus_client(self):
        return self.milvus_client

In [14]:
my_vdb_creator = VectorDBCreator("my_vector_db_01")
my_vdb_creator.create_collection("my_collection", dimension=768)

Collection 'my_collection' with dimension 768 created.


### Indexing

In [15]:
class DocumentIndexer:
    def __init__(self, milvus_client, collection_name: str):
        self.milvus_client = milvus_client
        self.collection_name = collection_name

    def index(self, data: list) -> dict:
        print(f"Inserting {len(data)} vectors into collection '{self.collection_name}'...")
        self.milvus_client.insert(collection_name=self.collection_name, data=data)
        stats = {"indexed_count": len(data)}
        print(f"Finished indexing: {stats['indexed_count']} vectors inserted.")
        return stats

In [16]:
my_document_indexer = DocumentIndexer(my_vdb_creator.get_milvus_client(), "my_collection")
my_document_indexer.index(prepared_embeddings)

Inserting 98 vectors into collection 'my_collection'...
Finished indexing: 98 vectors inserted.


{'indexed_count': 98}

### Retrieval

In [17]:
class DocumentRetriever:
    def __init__(self, milvus_client, collection_name: str, embedder):
        self.milvus_client = milvus_client
        self.collection_name = collection_name
        self.embedder = embedder

    def retrieve(self, query: str, k: int = 5) -> list:
        print(f"Processing query: {query}")
        query_embedding = self.embedder.get_embeddings(query)
        search_results = self.milvus_client.search(
            collection_name=self.collection_name,
            data=[query_embedding],
            limit=k,
            search_params={"metric_type": "IP", "params": {}},
            output_fields=["text", "headings", "page_info"]
        )
        results = []
        for res in search_results[0]:
            entity = res["entity"]
            results.append({
                "text": entity.get("text", ""),
                "headings": entity.get("headings", []),
                "page_info": entity.get("page_info", None),
                "distance": res["distance"]
            })
        return results

    def format_context(self, chunks: list) -> str:
        context_parts = []
        for i, chunk in enumerate(chunks):
            headings_field = chunk.get("headings", None)
            if isinstance(headings_field, list):
                heading_path = " > ".join(headings_field) if headings_field else "Document section"
            elif isinstance(headings_field, str):
                heading_path = headings_field or "Document section"
            else:
                heading_path = "Document section"
                
            page_ref = f"(Page {chunk.get('page_info')})" if chunk.get('page_info') else ""
            context_parts.append(
                f"EXCERPT {i+1} - {heading_path} {page_ref}:\n{chunk['text']}\n"
            )
        return "\n".join(context_parts)

In [18]:
my_document_retriever = DocumentRetriever(my_vdb_creator.get_milvus_client(), "my_collection", my_embedder)

retriever_results = my_document_retriever.retrieve("Wie viele Kalorien brauche ich bei wenig Bewegung?", k=3)
show_pretty_json(retriever_results)

Processing query: Wie viele Kalorien brauche ich bei wenig Bewegung?


```json
[
  {
    "text": "Um deinen Kalorienbedarf zu berechnen, musst du deinen Grundumsatz (BMR) und deinen Aktivit\u00e4tsfaktor kennen. Der BMR gibt an, wie viele Kalorien du im Ruhezustand verbrauchst, und der Aktivit\u00e4tsfaktor ber\u00fccksichtigt deine t\u00e4glichen Aktivit\u00e4ten.",
    "headings": "8.3 Wie berechne ich meinen Kalorienbedarf?",
    "page_info": "",
    "distance": 0.6846004724502563
  },
  {
    "text": "Der Kalorienbedarf eines Menschen h\u00e4ngt von verschiedenen Faktoren ab, wie Alter, Geschlecht, Gewicht, Gr\u00f6\u00dfe und Aktivit\u00e4tsniveau. Der Grundumsatz (BMR) ist die Anzahl der Kalorien, die der K\u00f6rper in Ruhe ben\u00f6tigt, um lebenswichtige Funktionen aufrechtzuerhalten. Der Gesamtenergieumsatz (TDEE) ber\u00fccksichtigt zus\u00e4tzlich die k\u00f6rperliche Aktivit\u00e4t.",
    "headings": "5.1 Kalorienbedarf",
    "page_info": "",
    "distance": 0.6833980083465576
  },
  {
    "text": "- \u00b7 Kalorien insgesamt: 1.700 kcal",
    "headings": "7.1.5 Gesamtkalorien und Makron\u00e4hrstoffe",
    "page_info": "",
    "distance": 0.6613255739212036
  }
]
```

In [19]:
formatted_context = my_document_retriever.format_context(retriever_results)
rprint(formatted_context)

EXCERPT 1 - 8.3 Wie berechne ich meinen Kalorienbedarf? :
Um deinen Kalorienbedarf zu berechnen, musst du deinen Grundumsatz (BMR) und deinen Aktivitätsfaktor kennen. Der 
BMR gibt an, wie viele Kalorien du im Ruhezustand verbrauchst, und der Aktivitätsfaktor berücksichtigt deine 
täglichen Aktivitäten.

EXCERPT 2 - 5.1 Kalorienbedarf :
Der Kalorienbedarf eines Menschen hängt von verschiedenen Faktoren ab, wie Alter, Geschlecht, Gewicht, Größe und 
Aktivitätsniveau. Der Grundumsatz (BMR) ist die Anzahl der Kalorien, die der Körper in Ruhe benötigt, um 
lebenswichtige Funktionen aufrechtzuerhalten. Der Gesamtenergieumsatz (TDEE) berücksichtigt zusätzlich die 
körperliche Aktivität.

EXCERPT 3 - 7.1.5 Gesamtkalorien und Makronährstoffe :
- · Kalorien insgesamt: 1.700 kcal

## Agent

### Agent-Tool für RAG

In [ ]:
# Configuration
class Config:
    OLLAPI_ENDPOINT_BASE = 'http://localhost:11434/v1'  # Base endpoint for Ollama
    OMODEL_LLAMA3D2 = 'llama3.2:latest'  # Model name

# Context class for agent state
class RAGContext(BaseModel):
    question: str = ""
    formatted_context: str = ""
    language: str = "English"  # Default language for responses

# Initialize the model - separate function for clarity
def create_llm_model():
    # Disable tracing to avoid messages about missing API keys
    set_tracing_disabled(True)
    
    # Create model with your endpoint
    return OpenAIChatCompletionsModel(
        model=config.OMODEL_LLAMA3D2,
        openai_client=AsyncOpenAI(
            base_url=config.OLLAPI_ENDPOINT_BASE, 
            api_key="fake-key"  # Using fake key as local endpoint doesn't require auth
        )
    )

# Function to extract information from prompt
async def extract_health_info(prompt: str) -> HealthData:
    data = {
        "model": config.OMODEL_LLAMA3D2,
        "prompt": prompt + "\nAntworte in JSON, passend zum Schema der HealthData-Klasse.",
        "stream": False,
        "format": HealthData.model_json_schema()
    }
    response = requests.post(config.OAPI_GENERATE_URL, json=data)
    response_json = response.json()
    response_data = json.loads(response_json["response"])
    return HealthData(**response_data)

# Diet comparison tool that uses DocumentRetriever
@function_tool
async def diet_comparison(
    context: RunContextWrapper[RAGContext], 
    query: str, 
    k: int = 4
    ) -> str:
    """
    Compares retrieved information about various diets.
    """

    try:
        # Store the question in context
        context.context.question = query
        
        # Use your existing document retriever
        retriever_results = my_document_retriever.retrieve(query, k=k)
        formatted_context = my_document_retriever.format_context(retriever_results)
        
        # Store formatted context in the agent context
        context.context.formatted_context = formatted_context
        
        return formatted_context
    except Exception as e:
        error_msg = f"Error retrieving documents: {str(e)}"
        print(error_msg)
        return error_msg
    
# BMI calculator
@function_tool
def bmi_calculator(height, weight) -> str:
    """
    Calculates the Body Mass Index (BMI) with given parameters.

    :param height: height in meters
    :param weight: weight in kilograms 
    :return: BMI
    """
    
    response = weight / float(height * height)
    response.raise_for_status()  # Ensure we catch any HTTP errors
    return response
    
# Calorie calculator
@function_tool
async def calorie_calculator(gender: str, weight_kg: float, height_cm: float, age: int, activity_level: str) -> dict:
    """
    Calculates Basal Metabolic Rate (BMR) and Total Daily Energy Expenditure (TDEE)
    using the Mifflin-St. Jeor equation.

    :param gender: "m" for male or "f" for female
    :param weight_kg: Weight in kilograms
    :param height_cm: Height in centimeters
    :param age: Age in years
    :param activity_level: One of the keys from `activity_factors`
    :return: Dictionary with BMR, TDEE, and activity factor used
    """
    activity_factors = {
        "sedentary": 1.2,        # little or no exercise
        "light": 1.375,          # light exercise 1–3 days/week
        "moderate": 1.55,        # moderate exercise 3–5 days/week
        "active": 1.725,         # hard exercise 6–7 days/week
        "very active": 1.9       # very intense physical job or training
    }

    gender = gender.lower()
    if gender == "m" or "male" or "man" or "mann" or "männlich":
        bmr = 10 * weight_kg + 6.25 * height_cm - 5 * age + 5
    elif gender == "f" or "female" or "woman" or "frau" or "weiblich":
        bmr = 10 * weight_kg + 6.25 * height_cm - 5 * age - 161
    else:
        raise ValueError("Invalid gender. Use 'm' for male or 'f' for female.")

    factor = activity_factors.get(activity_level.lower())
    if factor is None:
        raise ValueError(f"Invalid activity level: {activity_level}")

    tdee = bmr * factor

    return {
        "BMR": round(bmr, 2),
        "TDEE": round(tdee, 2),
        "Activity Factor": factor
    }

# Allergy information tool
@function_tool
async def allergy_check(
    context: RunContextWrapper[RAGContext], 
    query: str, 
    k: int = 4
    ) -> str:
    """
    Retrieves information about potential allergies in diets.
    """

    try:
        # Store the question in context
        context.context.question = query
        
        # Use your existing document retriever
        retriever_results = my_document_retriever.retrieve(query, k=k)
        formatted_context = my_document_retriever.format_context(retriever_results)
        
        # Store formatted context in the agent context
        context.context.formatted_context = formatted_context
        
        return formatted_context
    except Exception as e:
        error_msg = f"Error retrieving documents: {str(e)}"
        print(error_msg)
        return error_msg

# Create a single RAG agent - simplifying the design
# Using valid tool_use_behavior value
rag_agent = Agent[RAGContext](
    name="Food Advisory Assistant",
    instructions="""
    You are an assistant specialized in dietary advice and health information.
    
    WORKFLOW:
    1. When a user asks a question regarding diets, use the diet_comparison tool to retrieve relevant information
    2. When a user asks a question regarding BMI, use the bmi_calculator tool to retrieve relevant information
    3. When a user asks a question regarding calories, use the calorie_calculator tool to retrieve relevant information
    4. When a user asks a question regarding allergies, use the allergy_check tool to retrieve relevant information
    5. Carefully analyze the retrieved context
    6. Provide a clear, accurate answer based on the retrieved information
    7. If the information isn't available in the retrieved context, indicate this clearly
    
    Respond in English.
    Be helpful, accurate, and concise in your responses.
    """,
    tools=[diet_comparison, bmi_calculator, allergy_check, calorie_calculator],
    model=create_llm_model(),
    # Using a valid tool_use_behavior value
    tool_use_behavior="run_llm_again",
    # Set model settings to help with function calling
    model_settings=ModelSettings(
        temperature=0.1,  # Lower temperature for more deterministic responses
        tool_choice="auto"  # Auto tool choice
    )
)

# Manual process for RAG when the model doesn't handle function calling properly
async def manual_rag_process(question, k=1):
    """
    Manually execute the RAG process when the model doesn't properly use function calling.
    """
    try:
        # Direct call to retrieve documents
        retriever_results = my_document_retriever.retrieve(question, k=k)
        formatted_context = my_document_retriever.format_context(retriever_results)
        
        # Create a prompt with the retrieved context
        formatted_prompt = f"""
        Question: {question}

        Context from diet plans:
        {formatted_context}

        Based on the above context, please provide a concise and accurate answer to the question.
        """
        
        # Create context with the retrieved info
        context = RAGContext(
            question=question,
            formatted_context=formatted_context,
            language="English"
        )
        
        # Use a list for input items with the formatted prompt
        input_items = [{"content": formatted_prompt, "role": "user"}]
        
        # Run the model with this prompt
        run_result = await Runner.run(
            rag_agent,
            input=input_items,
            context=context
        )
        
        # Return the results
        return {
            "answer": run_result.final_output,
            "run_result": run_result,
            "context": context
        }
    except Exception as e:
        error_msg = f"Error in manual RAG process: {str(e)}"
        print(error_msg)
        return {"error": error_msg}

# Detailed trace function for better visibility into the agent process
def print_run_trace(run_result):
    print("\n== DETAILED AGENT RUN TRACE ==\n")
    
    # Print basic info
    print(f"Total items generated: {len(run_result.new_items)}")
    print(f"Model responses: {len(run_result.raw_responses)}")
    print("-" * 80)
    
    # Loop through each item and print details based on item type
    for i, item in enumerate(run_result.new_items):
        item_type = getattr(item, 'type', 'unknown')
        
        print(f"\n[STEP {i+1}: {item_type}]")
        
        if item_type == 'tool_call_item':
            # Print tool call details
            print(f"  Agent: {item.agent.name}")
            raw_item = item.raw_item
            print(f"  Tool called: {raw_item.name}")
            print(f"  Arguments: {raw_item.arguments}")
            
        elif item_type == 'tool_call_output_item':
            # Print tool output details
            print(f"  Agent: {item.agent.name}")
            print(f"  Output type: {item.raw_item.get('type', 'unknown')}")
            
            # Truncate long outputs for readability
            output = item.output
            if len(output) > 150:
                output = output[:150] + "..."
            print(f"  Output: {output}")
            
        elif item_type == 'message_output_item':
            # Print message details
            print(f"  Agent: {item.agent.name}")
            raw_item = item.raw_item
            
            # Extract and format content
            content = ""
            if hasattr(raw_item, 'content') and raw_item.content:
                for content_item in raw_item.content:
                    if hasattr(content_item, 'text'):
                        text = content_item.text
                        if len(text) > 150:
                            text = text[:150] + "..."
                        content = text
            
            print(f"  Role: {getattr(raw_item, 'role', 'unknown')}")
            print(f"  Content: {content}")
            
        else:
            # For any other item types
            print(f"  Item details: {item}")
            
        print("-" * 80)
    
    # Print token usage information if available
    print("\n== TOKEN USAGE ==")
    total_input_tokens = 0
    total_output_tokens = 0
    
    for i, response in enumerate(run_result.raw_responses):
        if hasattr(response, 'usage'):
            usage = response.usage
            input_tokens = getattr(usage, 'input_tokens', 0)
            output_tokens = getattr(usage, 'output_tokens', 0)
            total_tokens = getattr(usage, 'total_tokens', 0)
            
            print(f"Response {i+1}:")
            print(f"  Input tokens: {input_tokens}")
            print(f"  Output tokens: {output_tokens}")
            print(f"  Total tokens: {total_tokens}")
            
            total_input_tokens += input_tokens
            total_output_tokens += output_tokens
    
    print(f"\nTotal input tokens: {total_input_tokens}")
    print(f"Total output tokens: {total_output_tokens}")
    print(f"Grand total tokens: {total_input_tokens + total_output_tokens}")

# Main function with detection of function calling issues
async def ask(
    question: str, 
    document_retriever=None, 
    language="English"
    ) -> Dict:
    # Allow passing a document retriever if not defined globally
    global my_document_retriever
    if document_retriever is not None:
        my_document_retriever = document_retriever
    
    # Create context with specified language
    context = RAGContext(language=language)
    
    # First attempt: standard approach
    with trace("Food Advisory Assistant - Standard Approach"):
        # Format question to HealthData Object
        formatted_question = await extract_health_info(question)
        print("----- Extrahierte Daten -----")
        print(formatted_question.model_dump_json(indent=4))

        # Use a list for input items
        input_items = [{"content": str(formatted_question), "role": "user"}]
        
        # Run the agent
        run_result = await Runner.run(
            rag_agent,
            input=input_items,
            context=context
        )
    
    # Check if we got a proper answer or just a function call spec
    is_function_call_text = False
    if run_result.final_output:
        # Check if the output looks like a raw function call
        if run_result.final_output.startswith('{"name":') or \
           'diet_comparison' in run_result.final_output:
            print("INSIDE ")
            is_function_call_text = True
    
    # If the model returned a function call as text, use manual RAG approach
    if is_function_call_text:
        print("Detected function call specification in output. Switching to manual RAG process...")
        return await manual_rag_process(str(formatted_question), k=1)
    
    # Standard approach worked fine
    return {
        "answer": run_result.final_output,
        "run_result": run_result,
        "context": context
    }

# Example usage
async def demo_rag():
    question1 = "Was passt besser zu mir: Low Carb oder Mittelmeerdiät?"
    question2 = "Ich bin 1,75m groß und wiege 95kg - wie hoch ist mein BMI?"
    question3 = "Wie viele Kalorien brauche ich bei wenig Bewegung? Ich bin eine Frau, bin 30 Jahre, 170 cm groß, mit 65 Kilo."
    question4 = "Ich bin laktoseintolerant - welche Diäten schließen Milchprodukte aus?"
    question5 = "Kannst du mein Ziel 'fitter werden' klarer formulieren?"
    question6 = """
                I am 20 years old, male, weigh 85kg, and am 1.80m tall.
                I am lactose intolerant and enjoy eating vegan.
                My goal is to build more muscle, which is why I exercise moderately.
                For daily cooking, I plan to spend a maximum of 2 hours (120 minutes).
                Unfortunately, I am also diabetic, so I need to keep an eye on my blood sugar levels.
                Which type of diet can you recommend for me?"""
    question7 = """
                Ich bin 20 Jahre alt, männlich, wiege 85kg und bin 1,80m groß. 
                Ich bin laktoseintolerant und esse gerne vegan. 
                Mein Ziel ist es, mehr Muskeln aufzubauen, weshalb ich moderat Sport betreibe. 
                Für das tägliche Kochen plane ich maximal 2 Stunden (120 min) ein. 
                Leider bin ich auch Diabetiker, weshalb ich auch auf meinen Zuckerhaushalt achten muss. 
                Welche Diätform kannst du mir empfehlen?"""
    
    # question6 = "Welche Diäten gibt es?"
    # question7 = "Welche Diät empfiehlt sich, wenn ich abnehmen möchte?"
    # question8 = "Welche Diät is kalorienarm?"
    # question9 = "Welche Diäten beinhalten möglicherweise Allergien?"
    # question10 = "Was sind Makronährstoffe?"
    # question11 = "Welche Vitamine brauche ich für was und woher bekomme ich diese?"
    # question12 = "Welchen Tagesplan sollte ich bei der Paleo Diät einhalten?"

    result = await ask(question7, my_document_retriever)
    
    # Print the answer
    print("ANSWER:")
    print(result["answer"])
    
    # Use our detailed trace function
    print_run_trace(result['run_result'])
    
    # Print context preview
    context = result.get('context')
    if context and hasattr(context, 'formatted_context'):
        print("\n== RETRIEVED CONTEXT ==")
        print(context.formatted_context)
    
    return 'success' #result

# For Jupyter notebook execution
await demo_rag()

----- Extrahierte Daten -----
{
    "age": 20,
    "gender": "männlich",
    "weight": 85.0,
    "height": 1.8,
    "allergies": "Laktoseintoleranz",
    "eating_habits": "Vegan",
    "goal": "Muskelaufbau durch Sport",
    "activity_level": "moderat",
    "timeCooking": 120,
    "healthCondition": "Diabetes"
}
ANSWER:
I apologize for the mistake in my previous response. Based on your input, I will recalculate the calorie needs.

To determine your daily calorie needs, we need to consider your activity level, age, weight, and height. As a moderately active 20-year-old male who is 85 kg and 1.8 meters tall, your basal metabolic rate (BMR) can be estimated as follows:

BMR = 66 + (6.2 x weight in kg) + (12.7 x height in cm) - (6.8 x age in years)
= 66 + (6.2 x 85) + (12.7 x 180) - (6.8 x 20)
= 66 + 526 + 2280 - 136
= 2454 calories

Since you are moderately active, your daily calorie needs will be higher than your BMR. A commonly used estimate for daily calorie needs is the Harris-Benedict

'success'